In [ ]:
from disentangle.notebooks.naturemethods.splitting_notebooks_utils import get_config_from_saved_predictionfile_NM, get_input, get_gaussian_poisson_factors
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from disentangle.core.tiff_reader import load_tiff
pred_noisy2_fpath = "/group/jug/ashesh/data/paper_stats/Test_P64_G3-32-32_M50_Sk0/kth_0/pred_training_disentangle_2507_D33-M3-S0-L8_18_1.tif"
pred_noisy_fpath = "/group/jug/ashesh/data/paper_stats/Test_P64_G3-32-32_M50_Sk0/kth_4/pred_training_disentangle_2507_D33-M3-S0-L8_25_1.tif"

gt_fpath = '/group/jug/ashesh/kth_data/D33/kth0/gt_for_pred_training_disentangle_2507_D33-M3-S0-L8_11_1.tif'
pred_orig_fpath = '/group/jug/ashesh/data/paper_stats/Test_P64_G3-32-32_M50_Sk0/kth_0/pred_training_disentangle_2507_D33-M3-S0-L8_11_1.tif'

# gt_denoised_fpath = '/group/jug/ashesh/kth_data/D32/kth0/gt_for_pred_training_disentangle_2507_D32-M3-S0-L8_7_1.tif'
resolution = 45 #nm per pixel

gt = load_tiff(gt_fpath)
pred_orig = load_tiff(pred_orig_fpath)
pred_noisy = load_tiff(pred_noisy_fpath)
pred_noisy2 = load_tiff(pred_noisy2_fpath)
gt.shape, pred_orig.shape, pred_noisy.shape, pred_noisy2.shape

In [ ]:
plt.imshow(gt[:400,:400,0], cmap='gray')

In [ ]:
noisy_cfg = get_config_from_saved_predictionfile_NM(pred_noisy_fpath)
noisy_cfg2 = get_config_from_saved_predictionfile_NM(pred_noisy2_fpath)

gaussian_sigma, poisson_noise_factor = get_gaussian_poisson_factors(noisy_cfg)
noisy_inp = get_input(gt, pred_noisy.shape[-1], gaussian_sigma, poisson_noise_factor)[0]

gaussian_sigma2, poisson_noise_factor2 = get_gaussian_poisson_factors(noisy_cfg2)
noisy2_inp = get_input(gt, pred_noisy.shape[-1], gaussian_sigma2, poisson_noise_factor2)[0]

orig_inp = get_input(gt, pred_orig.shape[-1], None, -1)[0]

In [ ]:
outputdir = '/group/jug/ashesh/naturemethods/hhmi/'
fpath = os.path.join(outputdir, 'SNR_HHMI25_16bit.png')
print(fpath)

In [ ]:
from disentangle.analysis.plot_utils import clean_ax
from matplotlib_scalebar.scalebar import ScaleBar

wN = 600
wS = 600
# hs = np.random.randint(0, gt.shape[0] - wN)
# ws = np.random.randint(0, gt.shape[1] - wS)

save_to_file = True
_,ax = plt.subplots(figsize=(16,16),ncols=4,nrows=4)
for i in range(1,4):
    vmin = np.quantile(gt[...,i-1], 0.01)
    vmax = np.quantile(gt[...,i-1], 0.99)
    
    ax[0,i].imshow(gt[hs:hs+wN,ws:ws+wS,i-1], cmap='magma', vmin=vmin, vmax=vmax)
    ax[0,i].set_title(f'Ch{i}')
    ax[1,i].imshow(pred_orig[hs:hs+wN,ws:ws+wS,i-1], cmap='magma', vmin=vmin, vmax=vmax)
    # ax[1,i].set_title(f'Pred orig {i}')
    ax[2,i].imshow(pred_noisy[hs:hs+wN,ws:ws+wS,i-1], cmap='magma', vmin=vmin, vmax=vmax)
    # ax[2,i].set_title(f'Pred denoised {i}')
    ax[3,i].imshow(pred_noisy2[hs:hs+wN,ws:ws+wS,i-1], cmap='magma', vmin=vmin, vmax=vmax)

# set [0,0] off 
ax[0,0].axis('off')
ax[1,0].set_title('Input')
ax[1,0].imshow(orig_inp[hs:hs+wN,ws:ws+wS], cmap='magma')
ax[2,0].imshow(noisy_inp[hs:hs+wN,ws:ws+wS], cmap='magma')
ax[3,0].imshow(noisy2_inp[hs:hs+wN,ws:ws+wS], cmap='magma')

for i in range(3):
    scalebar = ScaleBar(resolution, 
                        "nm", 
                        # length_fraction=0.1, 
                        box_alpha=0.6, frameon=True, location='upper right', font_properties={'size':12})

    ax[1+i,0].add_artist(scalebar)

clean_ax(ax)
# reduce the space between the subplots
plt.subplots_adjust(wspace=0.02, hspace=0.02)
if save_to_file:
    plt.savefig(fpath, dpi=150, bbox_inches='tight')
    print(f'Saved to {fpath}')

In [ ]:
gaussian_sigma, poisson_noise_factor

In [ ]:
gaussian_sigma, poisson_noise_factor

In [ ]:
gaussian_sigma2, poisson_noise_factor2